# 02 - Coinbase Connector Exploration

**Goal**: Explore the Coinbase Advanced Trade public API to inform our production connector.

**Scope**:
- Test both `coinbase-advanced-py` SDK and raw `httpx` approaches
- Map Coinbase symbols to canonical pairs
- Parse responses into our `TopOfBook` dataclass
- Document rate limits, error handling, edge cases
- Decide: SDK wrapper vs raw httpx for production connector

**Supported Pairs** (from PROJECT_INSTRUCTIONS.md):
- BTC/USD, BTC/USDC
- LTC/USD, LTC/USDC, LTC/BTC
- SOL/USD, SOL/USDC, SOL/BTC

**Key Differences from Kraken**:
- Coinbase uses standard symbols: `BTC-USD` (hyphen-separated)
- Public endpoints exist (no auth needed for Phase 1)
- Rate limit: 10 req/sec by IP for public endpoints
- 1s cache on public endpoints (bypassable with `cache-control: no-cache`)
- SDK is synchronous (uses `requests`, not async)

**Lessons Applied** (from LESSONS_LEARNED.md):
- LL-001: Verify exact symbol format, don't assume
- LL-002: Document actual response shapes from live API, not just docs
- LL-003: Test rate limit behavior before building production connector
- LL-010: All prices/sizes via `to_decimal()`, never float


## 1. Setup


In [1]:
# Install dependencies (run once)
# !pip install coinbase-advanced-py httpx


In [2]:
import json
import sys
import time
from pprint import pprint

sys.path.insert(0, "../src")

# Our existing infrastructure — reuse, don't reimplement
from uscryptoarb.marketdata.topofbook import TopOfBook, tob_from_raw
from uscryptoarb.validation.guards import require_present
from uscryptoarb.venues.symbols import SymbolTranslator

## 2. Product Discovery

Coinbase uses hyphen-separated symbols: `BTC-USD`, `LTC-USDC`, etc.
Unlike Kraken (which uses `XBT` for BTC), Coinbase uses standard ticker symbols.

Let's first use the SDK to list available products, then verify our 8 target pairs exist.

**Critical check (DEC-001)**: USD and USDC must be distinct products.


In [3]:
from coinbase.rest import RESTClient

# No API keys needed for public endpoints
client = RESTClient()

# List all products to find our target pairs
products = client.get_public_products()
product_list = products.to_dict().get("products", [])
print(f"Total products available: {len(product_list)}")

# Show a sample product to understand the structure
if product_list:
    sample = product_list[0]
    print("\nSample product structure:")
    pprint(sample)

Total products available: 876

Sample product structure:
{'about_description': '',
 'alias': '',
 'alias_to': ['BTC-USDC'],
 'approximate_quote_24h_volume': '826666922.67',
 'auction_mode': False,
 'base_cbrn': '',
 'base_currency_id': 'BTC',
 'base_display_symbol': 'BTC',
 'base_increment': '0.00000001',
 'base_max_size': '3400',
 'base_min_size': '0.00000001',
 'base_name': 'Bitcoin',
 'cancel_only': False,
 'display_name': 'BTC-USD',
 'display_name_overwrite': '',
 'fcm_trading_session_details': None,
 'icon_color': '',
 'icon_url': '',
 'is_alpha_testing': False,
 'is_disabled': False,
 'limit_only': False,
 'market_cap': '',
 'mid_market_price': '',
 'new': False,
 'new_at': '2023-01-01T00:00:00Z',
 'post_only': False,
 'price': '68991.84',
 'price_increment': '0.01',
 'price_percentage_change_24h': '4.52993917304615',
 'product_cbrn': '',
 'product_id': 'BTC-USD',
 'product_type': 'SPOT',
 'product_venue': 'CBE',
 'quote_cbrn': '',
 'quote_currency_id': 'USD',
 'quote_display_sym

In [4]:
TARGET_PAIRS = [
    "BTC/USD",
    "BTC/USDC",
    "LTC/USD",
    "LTC/USDC",
    "LTC/BTC",
    "SOL/USD",
    "SOL/USDC",
    "SOL/BTC",
]


# Coinbase uses hyphen format: BTC-USD
def canonical_to_coinbase(pair: str) -> str:
    """Convert 'BTC/USD' -> 'BTC-USD'."""
    return pair.replace("/", "-")


# Build lookup of available products
available = {p["product_id"]: p for p in product_list}

print("Target pair availability on Coinbase:")
print("-" * 60)
found_pairs: dict[str, str] = {}
missing_pairs: list[str] = []

for canonical in TARGET_PAIRS:
    cb_symbol = canonical_to_coinbase(canonical)
    if cb_symbol in available:
        p = available[cb_symbol]
        status = p.get("status", "unknown")
        print(f"  ✅ {canonical:12s} -> {cb_symbol:10s} (status: {status})")
        found_pairs[canonical] = cb_symbol
    else:
        print(f"  ❌ {canonical:12s} -> {cb_symbol:10s} NOT FOUND")
        missing_pairs.append(canonical)

print(f"\nFound: {len(found_pairs)}/8 | Missing: {len(missing_pairs)}")
if missing_pairs:
    print(f"Missing pairs: {missing_pairs}")

Target pair availability on Coinbase:
------------------------------------------------------------
  ✅ BTC/USD      -> BTC-USD    (status: online)
  ✅ BTC/USDC     -> BTC-USDC   (status: online)
  ✅ LTC/USD      -> LTC-USD    (status: online)
  ✅ LTC/USDC     -> LTC-USDC   (status: online)
  ✅ LTC/BTC      -> LTC-BTC    (status: online)
  ✅ SOL/USD      -> SOL-USD    (status: online)
  ✅ SOL/USDC     -> SOL-USDC   (status: online)
  ✅ SOL/BTC      -> SOL-BTC    (status: online)

Found: 8/8 | Missing: 0


In [5]:
# DEC-001: USD and USDC must be distinct. Verify Coinbase treats them separately.
btc_usd = available.get("BTC-USD", {})
btc_usdc = available.get("BTC-USDC", {})

print("BTC-USD quote currency:", btc_usd.get("quote_currency_id", "N/A"))
print("BTC-USDC quote currency:", btc_usdc.get("quote_currency_id", "N/A"))
print(f"\nDistinct products: {btc_usd.get('product_id') != btc_usdc.get('product_id')}")

BTC-USD quote currency: USD
BTC-USDC quote currency: USDC

Distinct products: True


## 3. Symbol Mapping

Build the symbol map for the production connector.
Unlike Kraken's complex XBT/prefix system, Coinbase is straightforward: `BTC-USD`.

We reuse our existing `SymbolTranslator` from `venues/symbols.py`.


In [6]:
# Build the symbol map from discovered pairs
COINBASE_SYMBOL_MAP: dict[str, str] = {}
for canonical, cb_symbol in found_pairs.items():
    COINBASE_SYMBOL_MAP[canonical] = cb_symbol

print("COINBASE_SYMBOL_MAP = {")
for canonical, cb_sym in COINBASE_SYMBOL_MAP.items():
    print(f'    "{canonical}": "{cb_sym}",')
print("}")

# Build SymbolTranslator (reuse existing infrastructure)
coinbase_symbols = SymbolTranslator(venue="coinbase", canonical_to_venue=COINBASE_SYMBOL_MAP)

# Test round-trip translation
for canonical in found_pairs:
    venue_sym = coinbase_symbols.to_venue_symbol(canonical)
    back = coinbase_symbols.to_canonical(venue_sym)
    assert back == canonical, f"Round-trip failed: {canonical} -> {venue_sym} -> {back}"
    print(f"  {canonical:12s} <-> {venue_sym:10s} ✅")

COINBASE_SYMBOL_MAP = {
    "BTC/USD": "BTC-USD",
    "BTC/USDC": "BTC-USDC",
    "LTC/USD": "LTC-USD",
    "LTC/USDC": "LTC-USDC",
    "LTC/BTC": "LTC-BTC",
    "SOL/USD": "SOL-USD",
    "SOL/USDC": "SOL-USDC",
    "SOL/BTC": "SOL-BTC",
}
  BTC/USD      <-> BTC-USD    ✅
  BTC/USDC     <-> BTC-USDC   ✅
  LTC/USD      <-> LTC-USD    ✅
  LTC/USDC     <-> LTC-USDC   ✅
  LTC/BTC      <-> LTC-BTC    ✅
  SOL/USD      <-> SOL-USD    ✅
  SOL/USDC     <-> SOL-USDC   ✅
  SOL/BTC      <-> SOL-BTC    ✅


## 4. Best Bid/Ask (SDK)

The SDK provides `get_best_bid_ask()` (authenticated) and
`get_public_product_book()` (public, no auth).

Let's test the public endpoints since Phase 1 doesn't use API keys.

**Expected response shape** (from Coinbase docs):
```json
{
  "pricebooks": [{
    "product_id": "BTC-USD",
    "bids": [{"price": "69000.01", "size": "0.5"}],
    "asks": [{"price": "69000.50", "size": "0.3"}],
    "time": "2024-01-01T00:00:00Z"
  }]
}
```

Prices and sizes are **strings** — feeds directly into `to_decimal()` (LL-010).


In [7]:
# Public product book — no auth required
book = client.get_public_product_book(product_id="BTC-USD", limit=1)
book_dict = book.to_dict() if hasattr(book, "to_dict") else dict(book)

print("Product book response structure:")
pprint(book_dict)

Product book response structure:
{'last': '68991.835',
 'mid_market': '68991.835',
 'pricebook': {'asks': [{'price': '68991.84', 'size': '0.2545509'}],
               'bids': [{'price': '68991.83', 'size': '0.08827528'}],
               'product_id': 'BTC-USD',
               'time': '2026-02-14T01:03:22.968913Z'},
 'spread_absolute': '0.01',
 'spread_bps': '0.001449446775'}


In [8]:
# Fetch best bid/ask for all available target pairs
print("Best Bid/Ask for target pairs:")
print("-" * 80)

bbo_results: dict[str, dict] = {}

for canonical, cb_symbol in found_pairs.items():
    try:
        book = client.get_public_product_book(product_id=cb_symbol, limit=1)
        book_dict = book.to_dict() if hasattr(book, "to_dict") else dict(book)
        bbo_results[canonical] = book_dict

        # Extract BBO
        pricebook = book_dict.get("pricebook", book_dict)
        bids = pricebook.get("bids", [])
        asks = pricebook.get("asks", [])

        best_bid = bids[0] if bids else {"price": "N/A", "size": "N/A"}
        best_ask = asks[0] if asks else {"price": "N/A", "size": "N/A"}
        ts = pricebook.get("time", "N/A")

        print(
            f"  {canonical:12s} bid={best_bid['price']:>14s} ({best_bid['size']:>12s})"
            f"  ask={best_ask['price']:>14s} ({best_ask['size']:>12s})"
            f"  time={ts}"
        )
        time.sleep(0.15)  # Respect rate limits
    except Exception as exc:
        print(f"  {canonical:12s} ERROR: {exc}")
        bbo_results[canonical] = {"error": str(exc)}

Best Bid/Ask for target pairs:
--------------------------------------------------------------------------------
  BTC/USD      bid=      68991.83 (  0.08827528)  ask=      68991.84 (   0.2545509)  time=2026-02-14T01:03:22.968913Z
  BTC/USDC     bid=      68991.83 (  0.08415553)  ask=      68991.84 (  0.26147871)  time=2026-02-14T01:03:23.957369Z
  LTC/USD      bid=         55.23 (        0.02)  ask=         55.24 ( 38.65736404)  time=2026-02-14T01:03:22.368320Z
  LTC/USDC     bid=         55.23 (        0.02)  ask=         55.24 ( 25.96936404)  time=2026-02-14T01:03:24.386868Z
  LTC/BTC      bid=        0.0008 ( 63.01780043)  ask=      0.000801 ( 66.13789643)  time=2026-02-14T01:03:22.127453Z
  SOL/USD      bid=         84.84 ( 29.27437015)  ask=         84.85 ( 12.68108687)  time=2026-02-14T01:03:23.638915Z
  SOL/USDC     bid=         84.84 ( 29.27437015)  ask=         84.85 (  3.40376642)  time=2026-02-14T01:03:25.692456Z
  SOL/BTC      bid=     0.0012296 (       9.682)  ask=     0.0

In [9]:
# Inspect the exact response structure for one pair
# This is critical per LL-002: never assume response shapes
sample_pair = next(iter(found_pairs))
sample_data = bbo_results.get(sample_pair, {})
print(f"Detailed response for {sample_pair}:")
print(json.dumps(sample_data, indent=2, default=str))

Detailed response for BTC/USD:
{
  "pricebook": {
    "product_id": "BTC-USD",
    "bids": [
      {
        "price": "68991.83",
        "size": "0.08827528"
      }
    ],
    "asks": [
      {
        "price": "68991.84",
        "size": "0.2545509"
      }
    ],
    "time": "2026-02-14T01:03:22.968913Z"
  },
  "last": "68991.835",
  "mid_market": "68991.835",
  "spread_bps": "0.001449446775",
  "spread_absolute": "0.01"
}


## 5. Raw httpx Comparison

The `coinbase-advanced-py` SDK is synchronous (uses `requests`).
Our architecture requires async (Coding Rule 2.3).

Options:
1. Wrap SDK calls in `asyncio.to_thread()` — extra thread overhead, less control
2. Use raw `httpx` — consistent with Kraken connector, full async, header control

Let's test raw httpx against the same endpoints to compare.


In [10]:
import httpx

COINBASE_BASE_URL = "https://api.coinbase.com"

# Raw httpx — public product book
with httpx.Client(timeout=10.0) as http:
    resp = http.get(
        f"{COINBASE_BASE_URL}/api/v3/brokerage/market/product_book",
        params={"product_id": "BTC-USD", "limit": "1"},
    )
    resp.raise_for_status()
    raw_data = resp.json()

print("Raw httpx response structure:")
pprint(raw_data)

Raw httpx response structure:
{'last': '68991.835',
 'mid_market': '68991.835',
 'pricebook': {'asks': [{'price': '68991.84', 'size': '0.2614787'}],
               'bids': [{'price': '68991.83', 'size': '0.0840903'}],
               'product_id': 'BTC-USD',
               'time': '2026-02-14T01:03:25.152228Z'},
 'spread_absolute': '0.01',
 'spread_bps': '0.001449446775'}


In [11]:
# Compare: are the response structures identical?
print(
    "SDK response keys:",
    sorted(sample_data.keys()) if isinstance(sample_data, dict) else type(sample_data),
)
print(
    "httpx response keys:",
    sorted(raw_data.keys()) if isinstance(raw_data, dict) else type(raw_data),
)

# Check if SDK wraps/transforms the response
print("\nSDK modifies response:", sample_data != raw_data)

SDK response keys: ['last', 'mid_market', 'pricebook', 'spread_absolute', 'spread_bps']
httpx response keys: ['last', 'mid_market', 'pricebook', 'spread_absolute', 'spread_bps']

SDK modifies response: True


In [12]:
# Test cache bypass — important for arbitrage (stale data = missed opportunities)
with httpx.Client(timeout=10.0) as http:
    # Request 1: with cache
    r1 = http.get(
        f"{COINBASE_BASE_URL}/api/v3/brokerage/market/product_book",
        params={"product_id": "BTC-USD", "limit": "1"},
    )
    t1 = r1.json().get("pricebook", {}).get("time", "N/A")

    time.sleep(0.2)

    # Request 2: with no-cache header
    r2 = http.get(
        f"{COINBASE_BASE_URL}/api/v3/brokerage/market/product_book",
        params={"product_id": "BTC-USD", "limit": "1"},
        headers={"cache-control": "no-cache"},
    )
    t2 = r2.json().get("pricebook", {}).get("time", "N/A")

    print(f"With cache:    time={t1}")
    print(f"Without cache: time={t2}")
    print(f"Timestamps differ: {t1 != t2}")

With cache:    time=2026-02-14T01:03:25.152228Z
Without cache: time=2026-02-14T01:03:25.152228Z
Timestamps differ: False


In [13]:
import asyncio


async def fetch_coinbase_bbo_async(pairs: list[str]) -> dict:
    """Fetch BBO for multiple pairs using async httpx."""
    results = {}
    async with httpx.AsyncClient(timeout=10.0) as http:
        for pair_symbol in pairs:
            resp = await http.get(
                f"{COINBASE_BASE_URL}/api/v3/brokerage/market/product_book",
                params={"product_id": pair_symbol, "limit": "1"},
                headers={"cache-control": "no-cache"},
            )
            resp.raise_for_status()
            results[pair_symbol] = resp.json()
    return results


# Test async fetch
cb_symbols = list(found_pairs.values())
async_results = asyncio.run(fetch_coinbase_bbo_async(cb_symbols[:3]))
print(f"Async fetch returned {len(async_results)} results")
for sym, data in async_results.items():
    pricebook = data.get("pricebook", {})
    bids = pricebook.get("bids", [{}])
    asks = pricebook.get("asks", [{}])
    print(f"  {sym}: bid={bids[0].get('price', 'N/A')} ask={asks[0].get('price', 'N/A')}")

RuntimeError: asyncio.run() cannot be called from a running event loop

## 6. Parse to TopOfBook

Write a prototype parser that converts Coinbase responses into our `TopOfBook` dataclass.
Uses `tob_from_raw()` factory — validation happens at the boundary (DEC-003).

Key differences from Kraken parser:
- Coinbase bids/asks are dicts `{"price": "...", "size": "..."}`, not arrays
- Coinbase provides a `time` field (ISO 8601 string with timestamp)
- No XBT→BTC symbol translation needed


In [ ]:
from datetime import datetime


def parse_coinbase_product_book(
    raw: dict,
    canonical_pair: str,
    ts_local_ms: int,
) -> TopOfBook:
    """
    Parse Coinbase public product book response into TopOfBook.

    Args:
        raw: Raw JSON response from /market/product_book
        canonical_pair: Our canonical pair (e.g., 'BTC/USD')
        ts_local_ms: Local timestamp when data was received

    Returns:
        TopOfBook instance (validated via tob_from_raw)

    Raises:
        ValueError: If data is missing or invalid
    """
    pricebook = require_present(raw.get("pricebook", raw), f"{canonical_pair}.pricebook")
    bids = require_present(pricebook.get("bids"), f"{canonical_pair}.bids")
    asks = require_present(pricebook.get("asks"), f"{canonical_pair}.asks")

    if not bids:
        raise ValueError(f"{canonical_pair}: empty bids array")
    if not asks:
        raise ValueError(f"{canonical_pair}: empty asks array")

    best_bid = bids[0]
    best_ask = asks[0]

    # Parse exchange timestamp if available
    ts_exchange_ms = None
    time_str = pricebook.get("time")
    if time_str:
        try:
            dt = datetime.fromisoformat(time_str.replace("Z", "+00:00"))
            ts_exchange_ms = int(dt.timestamp() * 1000)
        except (ValueError, AttributeError):
            pass  # Fall back to None

    return tob_from_raw(
        venue="coinbase",
        pair=canonical_pair,
        ts_local_ms=ts_local_ms,
        ts_exchange_ms=ts_exchange_ms,
        bid_px=best_bid["price"],
        bid_sz=best_bid["size"],
        ask_px=best_ask["price"],
        ask_sz=best_ask["size"],
    )


# Test with live data
print("Parsing live data into TopOfBook:")
print("-" * 60)
for canonical in found_pairs:
    raw = bbo_results.get(canonical, {})
    if "error" in raw:
        print(f"  {canonical:12s} SKIPPED (fetch error)")
        continue
    try:
        ts_now = int(time.time() * 1000)
        tob = parse_coinbase_product_book(raw, canonical, ts_now)
        print(
            f"  {canonical:12s} bid={str(tob.bid_px):>14s} ask={str(tob.ask_px):>14s}"
            f"  spread={tob.ask_px - tob.bid_px}"
        )
    except Exception as exc:
        print(f"  {canonical:12s} PARSE ERROR: {exc}")

## 7. Rate Limits

Coinbase public endpoints: 10 req/sec by IP (per docs).
1s cache enabled for all public endpoints.

Let's test actual rate limit behavior (per LL-003: always test, don't just trust docs).


In [ ]:
# Test rapid successive requests
rate_results = []
rate_errors = []

with httpx.Client(timeout=10.0) as http:
    for i in range(15):
        try:
            start = time.time()
            resp = http.get(
                f"{COINBASE_BASE_URL}/api/v3/brokerage/market/product_book",
                params={"product_id": "BTC-USD", "limit": "1"},
                headers={"cache-control": "no-cache"},
            )
            elapsed = time.time() - start

            if resp.status_code == 429:
                rate_errors.append(f"Request {i + 1}: RATE LIMITED (429)")
                print(f"  Request {i + 1}: 429 RATE LIMITED after {elapsed:.3f}s")
                # Check retry-after header
                retry_after = resp.headers.get("retry-after", "N/A")
                print(f"    Retry-After: {retry_after}")
            else:
                resp.raise_for_status()
                rate_results.append(elapsed)
                print(f"  Request {i + 1}: {resp.status_code} in {elapsed:.3f}s")
        except Exception as e:
            rate_errors.append(str(e))
            print(f"  Request {i + 1}: ERROR - {e}")
        # No delay — testing the limit

if rate_results:
    print(f"\nSuccessful: {len(rate_results)} | Avg latency: {sum(rate_results) / len(rate_results):.3f}s")
if rate_errors:
    print(f"Rate limited/errors: {len(rate_errors)}")
    print(f"  First rate limit at request: {len(rate_results) + 1}")

In [ ]:
# Based on 10 req/sec limit, minimum interval should be 100ms
# But for safety and to leave headroom, recommend 150ms
# For our polling use case (all 8 pairs), one batch request is ideal

# Test batch BBO endpoint (fetches multiple pairs in one request)
with httpx.Client(timeout=10.0) as http:
    # Coinbase product_book only takes one product_id at a time
    # But best_bid_ask can take multiple product_ids
    start = time.time()
    resp = http.get(
        f"{COINBASE_BASE_URL}/api/v3/brokerage/market/best_bid_ask",
        params={"product_ids": list(found_pairs.values())},
    )
    elapsed = time.time() - start
    if resp.status_code == 200:
        batch_data = resp.json()
        pricebooks = batch_data.get("pricebooks", [])
        print(f"Batch BBO: {len(pricebooks)} pairs in {elapsed:.3f}s")
        for pb in pricebooks[:3]:
            pb_bids = pb.get("bids", [{}])
            pb_asks = pb.get("asks", [{}])
            print(
                f"  {pb['product_id']}: bid={pb_bids[0].get('price', 'N/A')}"
                f" ask={pb_asks[0].get('price', 'N/A')}"
            )
    else:
        print(f"Batch BBO failed: {resp.status_code} - {resp.text[:200]}")

print("\nRecommended RateLimiter interval: 150ms (10 req/s limit with headroom)")

## 8. Error Handling


In [ ]:
# Test various error scenarios to document response formats
with httpx.Client(timeout=10.0) as http:
    test_cases = [
        ("Invalid product", {"product_id": "FAKE-PAIR", "limit": "1"}),
        ("Empty product_id", {"product_id": "", "limit": "1"}),
        ("No product_id", {"limit": "1"}),
    ]

    for label, params in test_cases:
        try:
            resp = http.get(
                f"{COINBASE_BASE_URL}/api/v3/brokerage/market/product_book",
                params=params,
            )
            print(f"{label}: status={resp.status_code}")
            if resp.status_code != 200:
                print(f"  Body: {resp.text[:200]}")
        except Exception as e:
            print(f"{label}: EXCEPTION - {e}")
        time.sleep(0.2)

## 9. Product Details

Capture trading parameters we'll need for the production connector:
- Price/size precision (for rounding)
- Min order sizes
- Trading status


In [ ]:
# Capture product details for our target pairs
print("Product details for target pairs:")
print("-" * 80)
for canonical, cb_symbol in found_pairs.items():
    p = available.get(cb_symbol, {})
    print(f"\n{canonical} ({cb_symbol}):")
    print(f"  status:              {p.get('status', 'N/A')}")
    print(f"  base_currency:       {p.get('base_currency_id', 'N/A')}")
    print(f"  quote_currency:      {p.get('quote_currency_id', 'N/A')}")
    print(f"  base_min_size:       {p.get('base_min_size', 'N/A')}")
    print(f"  base_max_size:       {p.get('base_max_size', 'N/A')}")
    print(f"  quote_increment:     {p.get('quote_increment', 'N/A')}")
    print(f"  base_increment:      {p.get('base_increment', 'N/A')}")
    print(f"  price (from ticker): {p.get('price', 'N/A')}")

## 10. Summary & Findings

### Symbol Mapping
- Coinbase uses standard hyphen-separated symbols: `BTC-USD`, `SOL-USDC`
- No XBT/prefix weirdness like Kraken
- All 8 target pairs available: [fill in after running]

### USD ≠ USDC Verification (DEC-001)
- Confirmed: `BTC-USD` and `BTC-USDC` are separate products with distinct quote currencies

### Data Sources
| Endpoint | Auth | Batch | Latency | Timestamp |
| --- | --- | --- | --- | --- |
| `/market/product_book` | No | No (1 pair) | ~Xms | Yes (ISO 8601) |
| `/market/best_bid_ask` | No | Yes (multi) | ~Xms | Yes (ISO 8601) |

### Rate Limits
- 10 req/sec by IP for public endpoints
- 1s cache (bypassable with `cache-control: no-cache`)
- Recommend 150ms RateLimiter interval

### SDK vs httpx Decision
- SDK is synchronous only — doesn't fit our async architecture (Coding Rule 2.3)
- Raw httpx provides: async native, header control, consistent with Kraken connector
- **Recommendation**: Use raw `httpx` (same as Kraken connector)

### Response Structure
- Prices/sizes come as **strings** — clean for `to_decimal()` (no LL-010 risk)
- Bids/asks are `[{"price": "...", "size": "..."}]` dicts (not arrays like Kraken)
- Timestamps are ISO 8601 strings

### Production Connector Design
```python
# Recommended structure (mirrors Kraken connector):
connectors/coinbase/
    __init__.py
    client.py      # httpx-based async client
    parser.py      # parse_product_book(), parse_best_bid_ask() -> TopOfBook
    symbols.py     # COINBASE_SYMBOL_MAP, SymbolTranslator instance
```


In [ ]:
# Final symbol map for production
print("COINBASE_SYMBOL_MAP = {")
for canonical, cb_sym in COINBASE_SYMBOL_MAP.items():
    print(f'    "{canonical}": "{cb_sym}",')
print("}")

In [ ]:
# Reverse map (for parsing responses back to canonical)
COINBASE_TO_CANONICAL = {v: k for k, v in COINBASE_SYMBOL_MAP.items()}
print("\nCOINBASE_TO_CANONICAL = {")
for cb_sym, canonical in COINBASE_TO_CANONICAL.items():
    print(f'    "{cb_sym}": "{canonical}",')
print("}")